# Recomendador de libros en Goodreads

En este proyecto elaboré un recomendador de libros con base el perfil de un usuario en Goodreads. En esencia, el programa le sugiere a un usuario cuál es próximo libro que debe leer. Para recopilar la información que alimentará al recomendador, utilicé la API de Goodreads. 

#### Algunos aspectos que me gustaría incluir después:
- Incluir en el análisis los usuarios con forbidden access.
- Extraer automáticamente la lista de amigos de un usuario y que se incluyan en el análisis.

#### Mejoras pendientes a mi código:
- Quitar los libros duplicados con mayor precisión, pues puede llegar a ser difícil.
- Hay libros cuyo título está en inglés y que otros leyeron en español.
- Se colaron algunos libros que solo yo califiqué (debido a la transformación de cero a NaN)
- La fórmula para recomendar libros probablemente está equivocada. Temo que le esté dando mucho peso a ciertos libros y muy poco peso a libros de las personas con las que tengo correlación negativa. 


## I. Obtener información de varios usuarios

### 1.1 Función para extraer información de un usuario

Las siguientes funciones (en particular `extract_info`) permiten extraer toda la información del estante de libros leídos de un usuario. La información se arroja como un diccionario.

In [1]:
# Importar librerías
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
import numpy as np

In [2]:
# Claves para acceder al API
CONSUMER_KEY = 'PnnwwD2CW9RtBv6YqsjYw'
url = "https://www.goodreads.com/"

In [3]:
def shelf_info_author(user_id, shelf, page):
  
    """Arroja un BeautifulSoup object de una página del librero de un usuario"""
    
    time.sleep(1.1) # Para cumplir con los términos y condiciones de Goodreads
    info = requests.get(f'{url}/review/list?v=2&id={user_id}&shelf={shelf}&sort=title&page={page}&per_page=200&key={CONSUMER_KEY}')
    print('Status code: ', info.status_code)
    info_content = info.content
    soup = BeautifulSoup(info_content, 'lxml')
    return soup

In [4]:
def shelf_info(user_id, shelf, page):
  
    """Arroja un BeautifulSoup object de una página del librero de un usuario"""
    
    time.sleep(1.1) # Para cumplir con los términos y condiciones de Goodreads
    info = requests.get(f'{url}/review/list?v=2&id={user_id}&shelf={shelf}&sort=title&page={page}&per_page=200&key={CONSUMER_KEY}')
    print('Status code: ', info.status_code)
    info_content = info.content
    soup = BeautifulSoup(info_content, 'lxml')
    return soup

def paginas_por_estante(libros_en_estante):
    """Nos dice cuántas páginas se requieren para mostrar todos los libros del estante"""
    
    if libros_en_estante <= 200:
        return 1
    
    elif libros_en_estante <= 400:
        return 2
    
    elif libros_en_estante <= 600:
        return 3
    
    elif libros_en_estante <= 800:
        return 4
    
    elif libros_en_estante <= 1000:
        return 5
    
    elif libros_en_estante <= 1200:
        return 6

    elif libros_en_estante <= 1400:
        return 7
    
    elif libros_en_estante <= 1600:
        return 8
    
    elif libros_en_estante <= 1800:
        return 9
    
    elif libros_en_estante <= 2000:
        return 10
    
def extract_info(user_id, shelf, libros_en_estante):
    
    diccionario = {}
    
    # Definir variables de mis tags para ir agregando información mientras avanzo por varias páginas.
    isbn = []
    title = []
    num_pages = []
    publisher = []
    publication_year = []
    name = []
    rating = []
    average_rating = []
    ratings_count = []
    links = []
    
    #Determinar cuántas páginas consideraremos dentro del estante.
    paginas_to_scrap = paginas_por_estante(libros_en_estante)
    
    # Extraer información de cada una de las páginas
    for x in range(1, paginas_to_scrap+1):
        
        # Obtener código html en formato Soup
        page = shelf_info(user_id, shelf=shelf, page=x)
        
        # Obtener información de tags que no se repiten
        
        isbn_info = page.find_all(f'isbn')
        isbn_info = [elem.get_text() for elem in isbn_info]
        isbn.append(isbn_info)
        
        title_info = page.find_all(f'title')
        title_info = [elem.get_text() for elem in title_info]
        title.append(title_info)
        
        publisher_info = page.find_all(f'publisher')
        publisher_info = [elem.get_text() for elem in publisher_info]
        publisher.append(publisher_info)
        
        num_pages_info = page.find_all(f'num_pages')
        num_pages_info = [elem.get_text() for elem in num_pages_info]
        num_pages.append(num_pages_info)
        
        publication_year_info = page.find_all(f'publication_year')
        publication_year_info = [elem.get_text() for elem in publication_year_info]
        publication_year.append(publication_year_info)
        
        name_info = page.find_all(f'name')
        name_info = [elem.get_text() for elem in name_info]
        name.append(name_info)
        
        rating_info = page.find_all(f'rating')
        rating_info = [elem.get_text() for elem in rating_info]
        rating.append(rating_info)
        
        # Obtener información de tags que se repiten
        review_blocks = page.find_all('review')
        link_pattern = re.compile(r'www.goodreads.com.*')
        
        for review in review_blocks: 
            average_rating_info = review.find(f'average_rating').get_text()
            average_rating.append(average_rating_info)

            ratings_count_info = review.find(f'ratings_count').get_text()
            ratings_count.append(ratings_count_info)
            
            # Obtener links        
            if link_pattern.search(review.get_text()):
                link = re.findall(link_pattern, review.get_text()) 
                links.append(link)
    
            else: 
                print('Missing: ', review.title)
    
    # Aplanar variables con listas dentro de una lista
    isbn = [elem for listt in isbn for elem in listt]
    title = [elem for listt in title for elem in listt]
    num_pages = [elem for listt in num_pages for elem in listt]
    publisher = [elem for listt in publisher for elem in listt]
    publication_year = [elem for listt in publication_year for elem in listt]
    name = [elem for listt in name for elem in listt]
    rating = [elem for listt in rating for elem in listt]
    links = [elem[0] for elem in links]
    
    # Pasar todo a un diccionario
    diccionario[f'user_id'] = [user_id for x in range(0, len(isbn))]
    diccionario[f'shelf'] = [shelf for x in range(0, len(isbn))]
    
    diccionario[f'isbn'] = isbn
    diccionario[f'title'] = title
    diccionario[f'author'] = name
    diccionario[f'num_pages'] = num_pages
    diccionario[f'publication_year'] = publication_year
    diccionario[f'publisher'] = publisher
    diccionario[f'my_rating'] = rating
    
    diccionario[f'average_rating'] = average_rating
    diccionario[f'ratings_count'] = ratings_count
    diccionario[f'links'] = links
        
    return diccionario


### 1.2 Extraer información del usuario y sus contactos

A continuación, se extraerá la información del usuario 'Francisco Galán', así como la de diez de sus contactos.

In [5]:
# Información del usuario principal
francisco_galan = extract_info('40732498', 'read', 276)

Status code:  200
Status code:  200


In [6]:
# Información de nueve amigos
nicolas_papa = extract_info('85738242', 'read', 87)
fernando_lamoyi = extract_info('22410395', 'read', 104)
cova_sv = extract_info('72222895', 'read', 52)
mario_carballo = extract_info('18141767', 'read', 142)  
andrea_raisman = extract_info('63716476', 'read', 416)
vanessa_romero = extract_info('16421531', 'read', 96)  
maria_lama = extract_info('68889321', 'read', 137)

Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200


In [7]:
# Información de las tres personas a las que sigue
eduardo_rosas = extract_info('51214176', 'read', 158)  
stefan_schubert = extract_info('27953287', 'read', 142)
bill_gates = extract_info('23470', 'read', 228)

Status code:  200
Status code:  200
Status code:  200
Status code:  200


In [8]:
# Usuarios con forbidden access.
# srdjan_miletic = extract_info('11055732', 'read', 350)  
# pablo_staforini = extract_info('3093249', 'read', 1846)
# alvaro_migoya = extract_info('57665930', 'read', 63)  

Transformemos ahora la información a bases de datos en pandas:

In [56]:
data_francisco = pd.DataFrame(francisco_galan)
data_nicolas = pd.DataFrame(nicolas_papa)
data_fernando = pd.DataFrame(fernando_lamoyi)
data_cova_sv = pd.DataFrame(cova_sv)
data_mario = pd.DataFrame(mario_carballo)
data_andrea = pd.DataFrame(andrea_raisman)
data_vanessa = pd.DataFrame(vanessa_romero)
data_maria = pd.DataFrame(maria_lama)
data_eduardo = pd.DataFrame(eduardo_rosas)
data_stefan = pd.DataFrame(stefan_schubert)
data_bill = pd.DataFrame(bill_gates)

Ejemplo:

In [43]:
data_francisco.head(3)

,user_id,shelf,isbn,title,author,num_pages,publication_year,publisher,my_rating,average_rating,ratings_count,links
0,40732498,read,,1984,George Orwell,328,1950,New American Library,4,4.19,3274743,www.goodreads.com/book/show/5470.1984
1,40732498,read,0451457994,"2001: A Space Odyssey (Space Odyssey, #1)",Arthur C. Clarke,297,2000,Roc,3,4.15,266459,www.goodreads.com/book/show/70535.2001
2,40732498,read,0307465357,"The 4-Hour Workweek: Escape 9-5, Live Anywhere...",Timothy Ferriss,396,2009,Harmony,3,3.90,197425,www.goodreads.com/book/show/6444424-the-4-hour...


## II. Unir múltiples bases de datos en una sola

### 2.1 Crear columna con el score de cada persona

Al momento de unir las bases de datos, la columna `my_rating` se va a repetir en todas las bases de datos. Por ello, conviene cambiarle el nombre según el usuario.

In [45]:
contactos = ['francisco', 'nicolas', 'fernando', 'cova', 'mario', 'andrea', 'vanessa', 'maria', 'eduardo', 'stefan', 'bill']
data_contactos = [data_francisco, data_nicolas, data_fernando, data_cova_sv, data_mario, data_andrea, data_vanessa, data_maria, data_eduardo, data_stefan ,data_bill]
columnas_rating = [x + '_rating' for x in contactos]

In [46]:
# Modificar el nombre de las columnas
n = -1
for contacto in data_contactos:
    n += 1
    new_name = columnas_rating[n]
    contacto.rename(columns={'my_rating': new_name}, inplace=True)

In [47]:
# Checando que funcionó
print(data_vanessa.columns)

Index(['user_id', 'shelf', 'isbn', 'title', 'author', 'num_pages',
       'publication_year', 'publisher', 'vanessa_rating', 'average_rating',
       'ratings_count', 'links'],
      dtype='object')


### 2.2 Quitar columnas irrelevantes

In [48]:
columnas_irrelevantes = ['isbn', 'user_id', 'shelf', 'publisher', 'links', 'num_pages', 'publication_year']

In [49]:
# Quitando columnas
for contacto in data_contactos:
    for columna in columnas_irrelevantes:
        del contacto[columna]

Ejemplo: 

In [50]:
data_vanessa.head(3)

,title,author,vanessa_rating,average_rating,ratings_count
0,La hija única,Guadalupe Nettel,2,4.22,858
1,1984,George Orwell,5,4.19,3274743
2,21 Lessons for the 21st Century,Yuval Noah Harari,5,4.16,99101


### 2.3 Unir datasets

In [51]:
data_total = data_francisco.copy()

for dataset in data_contactos[1:]:
    data_total = data_total.merge(dataset, how="outer", on=['title', 'author', 'average_rating', 'ratings_count'])

In [52]:
data_total.head(3)

,title,author,francisco_rating,average_rating,ratings_count,nicolas_rating,fernando_rating,cova_rating,mario_rating,andrea_rating,vanessa_rating,maria_rating,eduardo_rating,stefan_rating,bill_rating
0,1984,George Orwell,4,4.19,3274743,4,4,NaN,4,4,5,4,NaN,NaN,2
1,"2001: A Space Odyssey (Space Odyssey, #1)",Arthur C. Clarke,3,4.15,266459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"The 4-Hour Workweek: Escape 9-5, Live Anywhere...",Timothy Ferriss,3,3.90,197425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.4 Eliminar duplicados (aún no logro hacer que funcione)

Hay algunos libros que están repetidos porque varía ligeramente el nombre del libro. Ejemplo:

In [54]:
match = "El mundo"
data_total.loc[data_total['title'].str.match(match), :]

,title,author,francisco_rating,average_rating,ratings_count,nicolas_rating,fernando_rating,cova_rating,mario_rating,andrea_rating,vanessa_rating,maria_rating,eduardo_rating,stefan_rating,bill_rating
161,El mundo de Sofía,Jostein Gaarder,4,3.93,207368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Podemos tomar los primeros 23 caracteres de un libro, verificar si coinciden con los de otro libro, y eliminar los duplicados:

In [58]:
data_test = data_total.copy()
data_test.shape

(1599, 15)

In [59]:
# # Loop sobre los libros de cada usuario
# for usuario in columnas_rating[1:]:
    
#     # Incluyo 20 filas de distancia para que no se eliminen versiones distintas en el mismo estante de un usuario.
#     row_num = 20
    
#     # Loop por cada fila
#     for i in range(len(data_test)):
#         row_num += 1 
#         for x in range(row_num, len(data_test)):
#             if (i in data_test.index) & (x in data_test.index):
                
#                 #Checar si hay un match
#                 if data_test.loc[i, 'title'][:25] == data_test.loc[x, 'title'][:25]:
                                        
#                     # Checar que el libro duplicado sea del usuario sobre el que se loopea
#                     if pd.notnull(data_test.loc[x, usuario]):
#                         print(f'Match | Row:{i} , Row:{x}')
#                         print(f"Original book: {data_test.loc[i, 'title']}")
#                         print(f"Duplicate book: {data_test.loc[x, 'title']}")
#                         print(f'Usuario: {usuario}\n')
                        
#                         # Fusionar las calificaciones en una fila y eliminar la fila duplicada.
#                         data_test.loc[i, usuario] = data_test.loc[x, usuario]
#                         data_test.drop(x, axis=0, inplace=True)

In [60]:
data_test.shape

(1599, 15)

### 2.5 Reiniciar índices

In [61]:
data_test = data_test.reset_index(drop=True)

### 2.6 Cambiar tipo de datos

In [62]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             1599 non-null   object
 1   author            1599 non-null   object
 2   francisco_rating  278 non-null    object
 3   average_rating    1599 non-null   object
 4   ratings_count     1599 non-null   object
 5   nicolas_rating    87 non-null     object
 6   fernando_rating   104 non-null    object
 7   cova_rating       53 non-null     object
 8   mario_rating      142 non-null    object
 9   andrea_rating     417 non-null    object
 10  vanessa_rating    96 non-null     object
 11  maria_rating      138 non-null    object
 12  eduardo_rating    159 non-null    object
 13  stefan_rating     145 non-null    object
 14  bill_rating       250 non-null    object
dtypes: object(15)
memory usage: 187.5+ KB


In [63]:
columnas = list(data_test.columns)
for columna in columnas[2:]:
    data_test[columna] = data_test[columna].astype('float')

In [64]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             1599 non-null   object 
 1   author            1599 non-null   object 
 2   francisco_rating  278 non-null    float64
 3   average_rating    1599 non-null   float64
 4   ratings_count     1599 non-null   float64
 5   nicolas_rating    87 non-null     float64
 6   fernando_rating   104 non-null    float64
 7   cova_rating       53 non-null     float64
 8   mario_rating      142 non-null    float64
 9   andrea_rating     417 non-null    float64
 10  vanessa_rating    96 non-null     float64
 11  maria_rating      138 non-null    float64
 12  eduardo_rating    159 non-null    float64
 13  stefan_rating     145 non-null    float64
 14  bill_rating       250 non-null    float64
dtypes: float64(13), object(2)
memory usage: 187.5+ KB


### 2.7 Limpiar valores de cero en las columnas

Hay veces en que un usuario lee un libro pero no le pone calificación. En esos casos, Goodreads le asigna un valor de cero al rating. Ejemplo:

In [65]:
data_test['francisco_rating'].value_counts()

3.0    101
2.0     67
4.0     57
5.0     24
1.0     20
0.0      9
Name: francisco_rating, dtype: int64

Mejor, cambiemos esos valores de cero a NaN para que no alteren otros cálculos:

In [66]:
for columna in columnas[2:]:
    data_test[columna] = data_test[columna].replace(0, np.NaN)

In [67]:
data_test['francisco_rating'].value_counts()

3.0    101
2.0     67
4.0     57
5.0     24
1.0     20
Name: francisco_rating, dtype: int64

## III. Elaborar sistema de recomendaciones

### 3.1 Obtener correlación con cada contacto

In [111]:
data = data_test.copy()
data = data.drop(['author', 'ratings_count'], axis=1)

In [112]:
corr_matrix = data.corr()
corr_matrix

,francisco_rating,average_rating,nicolas_rating,fernando_rating,cova_rating,mario_rating,andrea_rating,vanessa_rating,maria_rating,eduardo_rating,stefan_rating,bill_rating
francisco_rating,1.000000,0.274774,0.166667,-0.293640,NaN,NaN,0.367884,1.000000,-0.291748,0.461538,-0.262000,-0.555556
average_rating,0.274774,1.000000,0.580110,0.295531,0.416135,0.451819,0.343579,0.073824,0.231545,0.316929,0.343001,0.448602
nicolas_rating,0.166667,0.580110,1.000000,0.500000,NaN,NaN,0.176777,NaN,0.158777,-0.271163,NaN,NaN
fernando_rating,-0.293640,0.295531,0.500000,1.000000,NaN,0.217407,0.000000,0.327327,-0.707107,0.111803,-1.000000,-0.101015
cova_rating,NaN,0.416135,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mario_rating,NaN,0.451819,NaN,0.217407,NaN,1.000000,-0.178820,NaN,NaN,-0.405465,NaN,NaN
andrea_rating,0.367884,0.343579,0.176777,0.000000,NaN,-0.178820,1.000000,-0.456435,-0.134687,NaN,0.208514,0.428174
vanessa_rating,1.000000,0.073824,NaN,0.327327,NaN,NaN,-0.456435,1.000000,0.522233,NaN,NaN,-0.866025
maria_rating,-0.291748,0.231545,0.158777,-0.707107,NaN,NaN,-0.134687,0.522233,1.000000,-0.408248,-0.215562,NaN
eduardo_rating,0.461538,0.316929,-0.271163,0.111803,NaN,-0.405465,NaN,NaN,-0.408248,1.000000,-0.306186,NaN


In [130]:
corr_matrix['francisco_rating'].reset_index(level=0)

,index,francisco_rating
0,francisco_rating,1.000000
1,average_rating,0.274774
2,nicolas_rating,0.166667
3,fernando_rating,-0.293640
4,cova_rating,NaN
5,mario_rating,NaN
6,andrea_rating,0.367884
7,vanessa_rating,1.000000
8,maria_rating,-0.291748
9,eduardo_rating,0.461538


In [120]:
# Pasar las correlaciones del usuario 'Francisco Galán' a un solo dataframe
correlaciones = corr_matrix['francisco_rating']
correlaciones = correlaciones.drop('francisco_rating', axis=0)
correlaciones = correlaciones.reset_index(level=0)
correlaciones = correlaciones.reset_index(drop=True)
correlaciones = correlaciones.rename(columns={'index': 'usuario', 'francisco_rating': 'corr'})
correlaciones

,usuario,corr
0,average_rating,0.274774
1,nicolas_rating,0.166667
2,fernando_rating,-0.293640
3,cova_rating,NaN
4,mario_rating,NaN
5,andrea_rating,0.367884
6,vanessa_rating,1.000000
7,maria_rating,-0.291748
8,eduardo_rating,0.461538
9,stefan_rating,-0.262000


### 3.2 Estandarizar ratings

Al calificar, cada usuario tiene un estándar de qué es un buen libro y qué es un mal libro, y esta percepción no siempre coincide con una clasificación numérica. Por esa razón, conviene ver qué usuarios tienden a calificar más alto y quiénes, más bajo. 

In [138]:
avg_rating = data.describe().T['mean'][1:]
avg_rating

average_rating     3.965454
nicolas_rating     3.825581
fernando_rating    3.933333
cova_rating        4.850000
mario_rating       3.949153
andrea_rating      3.758454
vanessa_rating     3.810811
maria_rating       3.353383
eduardo_rating     4.120253
stefan_rating      3.699301
bill_rating        3.540000
Name: mean, dtype: float64

In [191]:
avg_rating = avg_rating.reset_index(level=0).rename(columns={'index': 'usuario'})
avg_rating

,usuario,mean
0,average_rating,3.965454
1,nicolas_rating,3.825581
2,fernando_rating,3.933333
3,cova_rating,4.850000
4,mario_rating,3.949153
5,andrea_rating,3.758454
6,vanessa_rating,3.810811
7,maria_rating,3.353383
8,eduardo_rating,4.120253
9,stefan_rating,3.699301


### 3.4 Fórmula para calcular qué tanto se recomienda un libro

In [175]:
# Quitar libros que ya leyó el usuario 'Francisco Galan'
f_data = data.loc[data['francisco_rating'].isnull(), :]
f_data.iloc[29:33, :]

,title,francisco_rating,average_rating,nicolas_rating,fernando_rating,cova_rating,mario_rating,andrea_rating,vanessa_rating,maria_rating,eduardo_rating,stefan_rating,bill_rating
298,The Fifth Risk,NaN,4.09,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299,Fight Club,NaN,4.18,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
300,Flow: The Psychology of Optimal Experience,NaN,4.10,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
301,Game Over: The Inside Story of the Greek Crisis,NaN,4.11,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [179]:
columnas = list(f_data.columns)[2:]

In [ ]:
# Fórmula para calcular el score de cada usuario
def score_usuario(usuario, libro):
    if #Rating del usuario para ese libro
        return 
    
    elif:
        libro_score = f_data.loc[f_data['title'] == libro, 'average_rating'].reset_index(drop=True)
        avg = avg_rating.loc[avg_rating[usuario] == 'average_rating', 'mean']
        corr = correlaciones.loc[correlaciones[usuario] == 'average_rating', 'corr']

        score = libro_score * corr / avg
        return score

### 3.5 Veredicto final